In [ ]:
import pandas as pd

# === 1. CSV einlesen ===
df_production = pd.read_csv(
    "Realisierte_Erzeugung_201501010000_202101010000_Jahr_1.csv",
    sep=";",
    dtype=str  # alles als String einlesen, damit wir die Zahlen selbst sauber parsen
)

# === 2. Spaltennamen bereinigen ===
df_production.columns = df_production.columns.str.strip()

# === 3. Jahreszahlen extrahieren ===
df_production["Jahr"] = pd.to_datetime(df_production["Datum von"], errors="coerce", dayfirst=True).dt.year

# === 4. Energie-Spalten identifizieren und manuell in Float umwandeln ===
energy_columns = [col for col in df_production.columns if "MWh" in col]
for col in energy_columns:
    df_production[col] = df_production[col].str.replace(".", "", regex=False)  # Punkt als Tausendertrennzeichen entfernen
    df_production[col] = df_production[col].str.replace(",", ".")  # Komma als Dezimalzeichen ersetzen
    df_production[col] = pd.to_numeric(df_production[col], errors="coerce")

# === 5. Gesamtproduktion berechnen ===
df_production["Gesamtproduktion_MWh"] = df_production[energy_columns].sum(axis=1)

# === 6. Ausgabe anzeigen ===
df_production[["Jahr", "Gesamtproduktion_MWh"] + energy_columns]

# === 6. Verbrauchs-CSV einlesen ===
df_consumption = pd.read_csv(
    "Realisierter_Stromverbrauch_201501010000_202101010000_Jahr_2.csv",
    sep=";",
    dtype=str
)
df_consumption.columns = df_consumption.columns.str.strip()
df_consumption["Jahr"] = pd.to_datetime(df_consumption["Datum von"], errors="coerce", dayfirst=True).dt.year

df_consumption["Netzlast [MWh] Berechnete Auflösungen"] = (
    df_consumption["Netzlast [MWh] Berechnete Auflösungen"]
    .str.replace(".", "", regex=False)
    .str.replace(",", ".")
    .astype(float)
)
df_consumption.rename(columns={"Netzlast [MWh] Berechnete Auflösungen": "Verbrauch_MWh"}, inplace=True)

# === 7. DataFrames mergen ===
df_combined = df_production[["Jahr", "Gesamtproduktion_MWh"]].merge(
    df_consumption[["Jahr", "Verbrauch_MWh"]], on="Jahr"
)


# === 8. Import berechnen ===
df_combined["Import_MWh"] = df_combined["Verbrauch_MWh"] - df_combined["Gesamtproduktion_MWh"]


df_combined

,Jahr,Gesamtproduktion_MWh,Verbrauch_MWh,Import_MWh
0,2015,5.390005e+08,5.002195e+08,-38780970.00
1,2016,5.413624e+08,5.030836e+08,-38278732.75
2,2017,5.105477e+08,5.056812e+08,-4866471.50
3,2018,5.422939e+08,5.092548e+08,-33039153.50
4,2019,5.213674e+08,4.973676e+08,-23999807.25
5,2020,5.036276e+08,4.853568e+08,-18270730.00
